In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import load_model
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [9]:
data = pd.read_csv('./data/iots.txt', delimiter = "\t")
data = data.iloc[:, :10]
combined_data = data
combined_data.head()

,Unnamed: 0,308511_speed,308511_flow,308512_speed,308512_flow,311903_speed,311903_flow,311974_speed,311974_flow,312010_speed
0,05/23/2024 00:00:00,66.1,17.0,63.8,43.0,66.5,99.0,70.7,108.0,70.7
1,05/23/2024 00:05:00,66.0,19.0,63.7,45.0,66.5,84.0,69.7,103.0,69.7
2,05/23/2024 00:10:00,65.9,19.0,63.7,43.0,66.4,92.0,69.4,106.0,69.4
3,05/23/2024 00:15:00,66.1,15.0,63.6,42.0,66.5,82.0,70.0,100.0,70.0
4,05/23/2024 00:20:00,66.0,17.0,63.7,41.0,66.4,96.0,69.7,98.0,69.7


In [10]:
combined_data.rename(columns={'Unnamed: 0': 'time'}, inplace=True)
# Set the 'time' column as the index
combined_data.set_index('time', inplace=True)

# Convert the index into a proper datetime format
combined_data.index = pd.to_datetime(data.index)

# Check the first few rows to confirm
combined_data.head()

,308511_speed,308511_flow,308512_speed,308512_flow,311903_speed,311903_flow,311974_speed,311974_flow,312010_speed
time,,,,,,,,,
2024-05-23 00:00:00,66.1,17.0,63.8,43.0,66.5,99.0,70.7,108.0,70.7
2024-05-23 00:05:00,66.0,19.0,63.7,45.0,66.5,84.0,69.7,103.0,69.7
2024-05-23 00:10:00,65.9,19.0,63.7,43.0,66.4,92.0,69.4,106.0,69.4
2024-05-23 00:15:00,66.1,15.0,63.6,42.0,66.5,82.0,70.0,100.0,70.0
2024-05-23 00:20:00,66.0,17.0,63.7,41.0,66.4,96.0,69.7,98.0,69.7


In [20]:
def create_sequences(data, seq_length):
    X, y = [], []

    for i in range(len(data) - seq_length):
        seq = data[i:(i + seq_length)]
        X.append(seq[:-1])
        y.append(seq[-1])

    return np.array(X), np.array(y)

data_308511 = combined_data[['308511_speed', '308511_flow']]
sclaer = MinMaxScaler(feature_range=(0, 1))
scaled_data = sclaer.fit_transform(data_308511)

# Choose a sequence length (e.g., 10 time steps)
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)
train_size = int(len(X) * 0.67)  # Taking 67% of data for training
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

In [28]:
# LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=2))  # for predicting both speed and flow
model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\yffan\Documents\VScodeProject\Python\smartcity-backend\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 9, 50)          │        10,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,902 (120.71 KB)

 Trainable params: 30,902 (120.71 KB)

 Non-trainable params: 0 (0.00 B)